In [ ]:
# default_exp Kaggle_TPS_Tutorial

# Getting Started Kaggle TPS Challenge with Tabular ML Toolkit

> A Tutorial to showcase usage of Tabular_ML_Toolkit library on Kaggle TPS Challenge Nov 2021

## Install

`pip install -U tabular_ml_toolkit`

## How to Best Use tabular_ml_toolkit

Start with your favorite model and then just simply create MLPipeline with one API.

*You can use MLPipeline to quickly train any model which supports scikit-lear fit and transform methods.*

*For example, Here we are using LogisticRegression from Scikit-Learn, on  [Kaggle TPS Challenge (Nov 2021) data](https://www.kaggle.com/c/tabular-playground-series-nov-2021/data)*

In [ ]:
from tabular_ml_toolkit.MLPipeline import *
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd
import numpy as np

# for visualizing pipeline
from sklearn import set_config
set_config(display="diagram")

In [ ]:
# Dataset file names and Paths
DIRECTORY_PATH = "/Users/pamathur/kaggle_datasets/tps_nov_2021/"
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
SAMPLE_SUB_FILE = "sample_submission.csv"

In [ ]:
# # create scikit-learn ml model
scikit_model = MLPClassifier(early_stopping=True, random_state=42)

In [ ]:
# createm ml pipeline for scikit-learn model
ml_pl = MLPipeline().prepare_data_for_training(
    train_file_path= DIRECTORY_PATH + TRAIN_FILE,
    test_file_path= DIRECTORY_PATH + TEST_FILE,
    #make sure to use right index and target column
    idx_col="id",
    target="target",
    model=scikit_model,
    random_state=42)

In [ ]:
ml_pl.spl

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                 ('scaler', StandardScaler())])),
                ('model', MLPClassifier(early_stopping=True, random_state=42))])

In [ ]:
ml_pl.dfl.create_train_valid(valid_size=0.2)

In [ ]:
# Quick check on dataframe shapes
print(f"X_train shape is {ml_pl.dfl.X_train.shape}" )
print(f"X_valid shape is {ml_pl.dfl.X_valid.shape}" )
print(f"y_train shape is {ml_pl.dfl.y_train.shape}")
print(f"y_valid shape is {ml_pl.dfl.y_valid.shape}")

X_train shape is (480000, 100)
X_valid shape is (120000, 100)
y_train shape is (480000,)
y_valid shape is (120000,)


In [ ]:
# Fit & Predict
ml_pl.spl.fit(ml_pl.dfl.X_train, ml_pl.dfl.y_train)
preds = ml_pl.spl.predict(ml_pl.dfl.X_valid)
preds_probs = ml_pl.spl.predict_proba(ml_pl.dfl.X_valid)[:, 1]

In [ ]:
# # quick check on predictions and predictions probabilities shape
# print(f"preds shape is {preds.shape}" )
# print(f"preds_probs shape is {preds_probs.shape}" )

preds shape is (120000,)
preds_probs shape is (120000,)


In [ ]:
# Metrics
auc = roc_auc_score(ml_pl.dfl.y_valid, preds_probs)
acc = accuracy_score(ml_pl.dfl.y_valid, preds)

print(f"AUC is : {auc} while Accuracy is : {acc} ")

AUC is : 0.7550680676073396 while Accuracy is : 0.7422916666666667 


Let's update the numerical scaler to MinMax

In [ ]:
from sklearn.preprocessing import MinMaxScaler

ml_pl.update_preprocessor(num_scaler=MinMaxScaler())
# check the udpated pipeline
ml_pl.spl

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                 ('scaler', MinMaxScaler())])),
                ('model', MLPClassifier(early_stopping=True, random_state=42))])

In [ ]:
# Fit & Predict
ml_pl.spl.fit(ml_pl.dfl.X_train, ml_pl.dfl.y_train)
preds = ml_pl.spl.predict(ml_pl.dfl.X_valid)
preds_probs = ml_pl.spl.predict_proba(ml_pl.dfl.X_valid)[:, 1]

In [ ]:
# Metrics
auc = roc_auc_score(ml_pl.dfl.y_valid, preds_probs)
acc = accuracy_score(ml_pl.dfl.y_valid, preds)
print(f"AUC is : {auc} while Accuracy is : {acc} ")

#### Let's Use K-Fold Training

In [ ]:
# check current pipeline
ml_pl.spl

In [ ]:
# fit and predict
sci_model_new_k_fold, sci_model_new_metrics_score = ml_pl.do_k_fold_training(n_splits=15, metrics=roc_auc_score)
print("mean metrics score:", np.mean(sci_model_new_metrics_score))

In [ ]:
print(f"Mean metrics_score is : {round(np.mean(metrics_score)*100,2)}")

In [ ]:
# predict
sci_model_new_preds = ml_pl.do_k_fold_prediction(k_fold=sci_model_new_k_fold)

print(sci_model_new_preds.shape)

In background `prepare_data_for_training` and `prepare_data_for_cv`  methods loads your input data into Pandas DataFrame, seprates X(features) and y(target).

Then `prepare_data_for_training` methods split X(features) into X_train, y_train, X_valid and y_valid DataFrames.
However, `prepare_data_for_cv`  method do not split but let's cross validation split internally X and y DataFrames.

Then both methods preprocess all numerical and categorical type data found in these DataFrames using scikit-learn pipelines. Then it bundle preprocessed data with your given model and return an MLPipeline object, this class instance has dataframeloader, preprocessor and scikit-lean pipeline instances, so you can call fit methods on X_train and y_train and predict methods on X_valid or X_test.

Please check detail documentation and source code for more details.

*NOTE: If you want to customize data and preprocessing steps you can do so by using `DataFrameLoader` and `PreProessor` classes. Check detail documentations for these classes for more options.*

#### Let's Use XGBosst on MLPipeline

*You can also use MLPipeline with XGBoost model, Just make sure to install XGBooost first depending upon your OS.*

*After that all steps remains same. Here is example using XGBRegressor with [Melbourne Home Sale price data](https://www.kaggle.com/estrotococo/home-data-for-ml-course)*

In [ ]:
#!pip install -U xgboost

In [ ]:
# Best way to install xgboost if you are on macosx and windows machine is using conda
# !conda install -c conda-forge xgboost

In [ ]:
# set xgb_params
xgb_params = {
#     'n_estimators': 1000,
#     'learning_rate': 0.01,
#     'max_depth': 9,
#     'booster': 'gbtree',
    'eval_metric': 'auc',
#     'tree_method': 'gpu_hist',
#     'predictor': 'gpu_predictor',
    'use_label_encoder': False,
    'random_state': 42
}

In [ ]:
from xgboost import XGBClassifier
# create xgb Classifier model
xgb_model = XGBClassifier(**xgb_params)

In [ ]:
# Update pipeline with xgb model
ml_pl.update_model(xgb_model)
ml_pl.spl

In [ ]:
# Fit & Predict
ml_pl.spl.fit(ml_pl.dfl.X_train, ml_pl.dfl.y_train)
preds = ml_pl.spl.predict(ml_pl.dfl.X_valid)
preds_probs = ml_pl.spl.predict_proba(ml_pl.dfl.X_valid)[:, 1]

In [ ]:
# Metrics
auc = roc_auc_score(ml_pl.dfl.y_valid, preds_probs)
acc = accuracy_score(ml_pl.dfl.y_valid, preds)

print(f"AUC is : {auc} while Accuracy is : {acc} ")

#### Let's Use K-Fold Training for xgb model

In [ ]:
# update pipeline with new model
ml_pl.model = scikit_model_new

In [ ]:
# fit and predict
xgb_k_fold, xgb_k_metrics_score = ml_pl.do_k_fold_training(n_splits=15, metrics=roc_auc_score)
print("mean metrics score:", np.mean(xgb_k_metrics_score))

In [ ]:
print(f"Mean metrics_score is : {round(np.mean(xgb_k_metrics_score)*100,2)}")

In [ ]:
# predict
xgb_preds = ml_pl.do_k_fold_prediction(k_fold=xgb_k_fold)

print(xgb_preds.shape)

In [ ]:
# take weighted average of both k-fold models predictions
final_preds = ((0.7 * sci_model_new_preds) + (0.3* xgb_pred)) / 2
print(final_preds.shape)

#### Create Kaggle Predictions

In [ ]:
sub = pd.read_csv(DIRECTORY_PATH + SAMPLE_SUB_FILE)
sub['target'] = final_preds
sub.to_csv('submission.csv', index=False)

In [ ]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()